In [1]:
import requests
from requests import get  # to make GET request
import json
import os
import glob
import random
import time
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord


def json2python(data):
    try:
        return json.loads(data)
    except:
        pass
    return None

def download(url, file_name):
    # open in binary mode
    with open(file_name, "wb") as file:
        # get request
        response = get(url)
        # write to file
        file.write(response.content)
        
def astrometry_by_astrometrynet(DIR, prefix, apikey):
    
    """
    DIR - directory where files are saved
    prefix - prefix of files you want to proccess
    apikey - apikey which you recieved after registration on astrometry.net
    """

    os.chdir(DIR) # change working directory
    files = os.listdir(os.curdir) # search all files current directory
    fitses = [file for file in files if file[:len(prefix)] == prefix] # choose files with defined prefix
    
    url_start = 'http://nova.astrometry.net/api/login' 
    url_api = 'http://nova.astrometry.net/api/'    

    R = requests.post(url_start, data={'request-json': json.dumps({"apikey": apikey})}) # login in the astronometry.net
    print(R.text) # if all went well you must get session number you'll recieve this:
              # {"status": "success", "message": "authenticated user: crucifer.troll@gmail.com", "session": "t4jawwmumeb9hyc8vly6rqovp51lb0zf"}

    R_p = json2python(R.text) 
    session = R_p['session'] # saving session number for further operations

    for fn in fitses:  # cycle for every file in fitses
        hdu = fits.open(fn) 
        head = hdu[0].header # read fits header for image coordinates
        c = SkyCoord(head['RA'], head['Dec'], unit=(u.hourangle, u.deg)) 
        args = {"publicly_visible": "n", "allow_modifications": "n", "session": session, "allow_commercial_use": "n",
            "center_ra": c.ra.deg, "center_dec": c.dec.deg, "radius": 0.2, "scale_units": 'arcsecperpix', "scale_est": 0.6}
        #args consist of arguments which you can send to astrometry.net full list you can find here:
        #http://astrometry.net/doc/net/api.html
        #I've set coordinates of the image center and image scale for RTT-150
    
        f = open(fn, 'rb') 
        file_args = (fn, f.read()) # reading fits file usual way to send it 
        json1 = json.dumps(args)

         # Because we're sending a file, we need format a multipart/form-data

    
        boundary_key = ''.join([random.choice('0123456789') for i in range(19)])
        boundary = '===============%s==' % boundary_key
        headers = {'Content-Type':'multipart/form-data; boundary="%s"' % boundary}
        data_pre = (    '--' + str(boundary) + '\n' +
                    'Content-Type: text/plain\r\n' +
                    'MIME-Version: 1.0\r\n' +
                    'Content-disposition: form-data; name="request-json"\r\n' +
                    '\r\n' +
                    json1 + '\n' +
                    '--' + str(boundary) + '\n' +
                    'Content-Type: application/octet-stream\r\n' +
                    'MIME-Version: 1.0\r\n' +
                    'Content-disposition: form-data; name="file"; filename="%s"' % file_args[0] +
                    '\r\n' + '\r\n')
        data_post = ('\n' + '--' + boundary + '--\n')
    
        data = data_pre.encode() + file_args[1] + data_post.encode() # here is the formed data to send

        R = requests.post(url=url_api+'/upload', headers=headers, data=data) # the longest part of the code, uploading file to server
        print(R.text)
        R_p = json2python(R.text)
        req_id = R_p['subid'] # we recieved submission ID
        R = requests.post(url_api+'submissions/'+str(req_id), data={'request-json': json.dumps({"session": session})})    
        print(R.text,'\t')
        R_p = json2python(R.text)
        print('SUBID:%s'%req_id, 'Wait a minute to upload the file') 
        jobid = R_p['jobs']
    
    
        while len(R.text) < 220:    
            time.sleep(30)
            R = requests.post(url_api+'submissions/'+str(req_id), data={'request-json': json.dumps({"session": session})})
            R_p = json2python(R.text)
            jobid = R_p['jobs'] # when file is uploaded we recieve JOBID
            print('30 seconds past')
        
        print('JOBID:%s'%jobid[0], 'Wait few minutes while astrometry.net working')
    

        status_wcs = ''
        while status_wcs != 'success':   
            time.sleep(30)
            print('30 seconds past')
            R = requests.post('http://nova.astrometry.net/api/jobs/'+str(jobid[0])+'/info/', data={'request-json': json.dumps({"session": session})})    
            print(R.text, '\t')
            R_p = json2python(R.text)
            status_wcs = R_p['status'] # when field resolved we get a success and can download file
            
        
        download('http://nova.astrometry.net/new_fits_file/'+str(jobid[0]), 'nwc_'+fn) #downloading file with new prefix "nwc"
        print('nwc_'+fn+' is ready') #end of the cycle, start again

In [2]:
DIR = 'e:/Observations/BTA/s091021/MTDra_phot/' 
prefix = 's7'
apikey = 'gfrgdeojaijcpsli'
astrometry_by_astrometrynet(DIR, prefix, apikey)

{"status": "success", "message": "authenticated user: crucifer.troll@gmail.com", "session": "xl8t4zd1bufi1pi6eqtx97ifge9kapif"}
{"status": "success", "subid": 5625335, "hash": "666f5cf24a7beade1621758597b2ab11f2c43449"}
{"user": 13779, "processing_started": "None", "processing_finished": "None", "user_images": [], "images": [], "jobs": [], "job_calibrations": []} 	
SUBID:5625335 Wait a minute to upload the file
30 seconds past
30 seconds past
30 seconds past
JOBID:6326018 Wait few minutes while astrometry.net working
30 seconds past
{"objects_in_field": [], "machine_tags": [], "tags": [], "status": "success", "original_filename": "s7070401.fts", "calibration": {"ra": 281.74579084555086, "dec": 55.641806088095805, "radius": 0.07019931478626189, "pixscale": 0.3490200810001044, "orientation": -14.167828168041325, "parity": -1.0}} 	
nwc_s7070401.fts is ready
{"status": "success", "subid": 5625337, "hash": "45e900b055711a26f8de20b68dd766ad89909fea"}
{"user": 13779, "processing_started": "No

{"status": "success", "subid": 5625373, "hash": "14791f8abf3315bcdc12ff8fa9dde2cd00b753a3"}
{"user": 13779, "processing_started": "None", "processing_finished": "None", "user_images": [], "images": [], "jobs": [], "job_calibrations": []} 	
SUBID:5625373 Wait a minute to upload the file
30 seconds past
30 seconds past
JOBID:6326056 Wait few minutes while astrometry.net working
30 seconds past
{"objects_in_field": [], "machine_tags": [], "tags": [], "status": "success", "original_filename": "s7070412.fts", "calibration": {"ra": 281.7468217400173, "dec": 55.64201386953386, "radius": 0.07015617420656847, "pixscale": 0.34880559274384676, "orientation": -14.20531581304033, "parity": -1.0}} 	
nwc_s7070412.fts is ready
{"status": "success", "subid": 5625375, "hash": "270f2040518c2e2d5af2739a0f0efad8e3c8dd0e"}
{"user": 13779, "processing_started": "None", "processing_finished": "None", "user_images": [], "images": [], "jobs": [], "job_calibrations": []} 	
SUBID:5625375 Wait a minute to upload t

30 seconds past
30 seconds past
JOBID:6326090 Wait few minutes while astrometry.net working
30 seconds past
{"objects_in_field": [], "machine_tags": [], "tags": [], "status": "success", "original_filename": "s7070435.fts", "calibration": {"ra": 281.7460631906949, "dec": 55.64325828202121, "radius": 0.07018996781743728, "pixscale": 0.348973609324046, "orientation": -14.26098351492906, "parity": -1.0}} 	
nwc_s7070435.fts is ready
{"status": "success", "subid": 5625412, "hash": "3c7d93ce146f17c16de296f7acf1f2a916fdae34"}
{"user": 13779, "processing_started": "None", "processing_finished": "None", "user_images": [], "images": [], "jobs": [], "job_calibrations": []} 	
SUBID:5625412 Wait a minute to upload the file
30 seconds past
30 seconds past
JOBID:6326092 Wait few minutes while astrometry.net working
30 seconds past
{"objects_in_field": [], "machine_tags": [], "tags": [], "status": "success", "original_filename": "s7070439.fts", "calibration": {"ra": 281.746056188722, "dec": 55.64329215